In [14]:
from transformers import pipeline
import pandas as pd
from ipywidgets import widgets, Button, Output
from IPython.display import display

In [58]:
# top_k=1로 설정하여 하나의 답변만 반환하도록 설정
qa_model = pipeline(
    task='table-question-answering', 
    model='dsba-lab/koreapas-finetuned-korwikitq', 
    top_k=1)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [59]:
weather = pd.read_csv('extremum_20240911111909.csv',encoding='euc-kr')
weather

,\t\t지점번호,지점명,일시,평균기온(℃),최고기온(℃),\t최고기온시각,최저기온(℃),최저기온시각일교차,Unnamed: 8
0,\t\t108,서울,2020.1.1,-2.2,0.3,14:57,-6.5,0:01,6.8
1,\t\t108,서울,2020.1.2,1.0,3.8,15:00,-0.7,0:01,4.5
2,\t\t108,서울,2020.1.3,-0.1,4.6,15:47,-3.4,8:07,8.0
3,\t\t108,서울,2020.1.4,1.2,6.1,14:50,-2.8,5:12,8.9
4,\t\t108,서울,2020.1.5,1.3,6.6,14:53,-3.2,8:21,9.8
...,...,...,...,...,...,...,...,...,...
1718,\t,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1719,\t,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1720,\t\t,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1721,\t,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
# Step 2: 데이터 확인 및 전처리 (모든 데이터를 문자열로 변환)
weather = weather.astype(str).fillna("")

In [61]:
weather = weather.dropna()

In [62]:
weather.rename(columns={'Unnamed: 8': '일교차','평균기온(℃)':'평균기온','최고기온(℃)':'최고기온','최저기온(℃)':'최저기온'}, inplace=True)

In [63]:
weather.columns = weather.columns.str.replace('\t','')
weather

,지점번호,지점명,일시,평균기온,최고기온,최고기온시각,최저기온,최저기온시각일교차,일교차
0,\t\t108,서울,2020.1.1,-2.2,0.3,14:57,-6.5,0:01,6.8
1,\t\t108,서울,2020.1.2,1.0,3.8,15:00,-0.7,0:01,4.5
2,\t\t108,서울,2020.1.3,-0.1,4.6,15:47,-3.4,8:07,8.0
3,\t\t108,서울,2020.1.4,1.2,6.1,14:50,-2.8,5:12,8.9
4,\t\t108,서울,2020.1.5,1.3,6.6,14:53,-3.2,8:21,9.8
...,...,...,...,...,...,...,...,...,...
1718,\t,nan,nan,nan,nan,nan,nan,nan,nan
1719,\t,nan,nan,nan,nan,nan,nan,nan,nan
1720,\t\t,nan,nan,nan,nan,nan,nan,nan,nan
1721,\t,nan,nan,nan,nan,nan,nan,nan,nan


In [64]:
weather = weather.iloc[:,2:]
weather

,일시,평균기온,최고기온,최고기온시각,최저기온,최저기온시각일교차,일교차
0,2020.1.1,-2.2,0.3,14:57,-6.5,0:01,6.8
1,2020.1.2,1.0,3.8,15:00,-0.7,0:01,4.5
2,2020.1.3,-0.1,4.6,15:47,-3.4,8:07,8.0
3,2020.1.4,1.2,6.1,14:50,-2.8,5:12,8.9
4,2020.1.5,1.3,6.6,14:53,-3.2,8:21,9.8
...,...,...,...,...,...,...,...
1718,nan,nan,nan,nan,nan,nan,nan
1719,nan,nan,nan,nan,nan,nan,nan
1720,nan,nan,nan,nan,nan,nan,nan
1721,nan,nan,nan,nan,nan,nan,nan


In [70]:
import pandas as pd
from transformers import pipeline

# Step 1: 데이터 로드
file_path = './pre_weather.csv'
weather = pd.read_csv(file_path)

# Step 2: 데이터 확인 및 전처리 (모든 데이터를 문자열로 변환)
weather = weather.astype(str).fillna("")

# Step 3: 테이블 크기를 확인하고 필요한 열만 남기기 (예: '일시', '일교차')
weather = weather[['일시', '일교차']]

# Step 4: 모델 로드 (Hugging Face의 TAPAS 한국어 미세조정 모델)
qa_model = pipeline(task='table-question-answering', model='dsba-lab/koreapas-finetuned-korwikitq',top_k=1)

/opt/anaconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [71]:
# Step 5: 위젯 생성 (질문 입력창과 버튼)
question_input = widgets.Text(
    value='',
    placeholder='질문을 입력하세요 (예: 2020년 1월 1일 일교차는?)',
    description='질문:',
    disabled=False
)

submit_button = Button(description="질문하기")
output_area = Output()

In [72]:

# Step 6: 예외 처리 및 모델 실행 함수 정의
def ask_question(b):
    with output_area:
        output_area.clear_output()  # 이전 결과를 지움
        query = question_input.value
        
        # 질문 형식 확인 및 데이터 범위 체크
        try:
            if not query:
                raise ValueError("질문을 입력하세요.")
            
            # 모델 실행 (top_k=1로 설정했으므로 하나의 답변만 반환)
            answer = qa_model(table=weather, query=query)
            
            # 출력 형식 수정
            print(f"질문: {query}")
            print(f"답변: {answer['answer']}")
        
        except ValueError as e:
            print(f"오류 발생: {e}")
        except Exception as e:
            print(f"모델 처리 중 오류 발생: {e}")



In [73]:

# Step 7: 버튼 클릭 시 질문을 실행하는 기능 연결
submit_button.on_click(ask_question)

# Step 8: UI 디스플레이
display(question_input, submit_button, output_area)

Text(value='', description='질문:', placeholder='질문을 입력하세요 (예: 2020년 1월 1일 일교차는?)')

Button(description='질문하기', style=ButtonStyle())

Output()